In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
train_df #ignore the warnings
X = train_df.drop(columns=['Unnamed: 0','ID','class'])
y = train_df['class']

imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [ ]:
import numpy as np

#summary features
ndvi_vals = X_imputed.values
ndvi_mean = np.mean(ndvi_vals, axis=1)
ndvi_std  = np.std(ndvi_vals, axis=1)
ndvi_max  = np.max(ndvi_vals, axis=1)
ndvi_min  = np.min(ndvi_vals, axis=1)
ndvi_range = ndvi_max - ndvi_min

# Add features
X_features = X_imputed.copy()
X_features['ndvi_mean']  = ndvi_mean
X_features['ndvi_std']   = ndvi_std
X_features['ndvi_max']   = ndvi_max
X_features['ndvi_min']   = ndvi_min
X_features['ndvi_range'] = ndvi_range

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(multi_class='multinomial', solver='lbfgs', C=1.0, class_weight='balanced', max_iter=1000)
)
pipeline.fit(X_features, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    multi_class='multinomial'))])

In [ ]:
pipeline.fit(X_features, y)

test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

In [ ]:
X_test = test_df.drop(columns=['Unnamed: 0','ID'])
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

#on test set

ndvi_vals_test = X_test_imputed.values.astype(float)
X_test_imputed['ndvi_mean']  = np.mean(ndvi_vals_test, axis=1)
X_test_imputed['ndvi_std']   = np.std(ndvi_vals_test, axis=1)
X_test_imputed['ndvi_max']   = np.max(ndvi_vals_test, axis=1)
X_test_imputed['ndvi_min']   = np.min(ndvi_vals_test, axis=1)
X_test_imputed['ndvi_range'] = X_test_imputed['ndvi_max'] - X_test_imputed['ndvi_min']

#test set
predictions = pipeline.predict(X_test_imputed)

# submission
submission = pd.DataFrame({'ID': test_df['ID'], 'class': predictions})
submission.to_csv('submission.csv', index=False)